In [253]:
import os
from zipfile import is_zipfile, ZipFile
from pickle import load, dump
import matplotlib.pyplot as plt
from numpy import array, stack, zeros
import torch 
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn 
from torch import optim 
# from torchsummary import summary

## Unzip precoder dataset

In [254]:
dir_name = 'datasets'
if os.path.exists(dir_name):
    print("Data has already been unzipped.")
else:
    if os.path.exists(f'{dir_name}.zip'):
        if is_zipfile(f'{dir_name}.zip'):
            with ZipFile(f'{dir_name}.zip', 'r') as zip_data_obj:
                zip_data_obj.extractall()
        else:
            print('File exists, but is not a zip archive.')
    else:
        print('datasets.zip was not found.')



datasets.zip was not found.


## Load precoder dataset

In [255]:
def load_dataset(dir_name, fading, num_ant, SNR_dB, bit_codebook):

    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    file_name = f'{dir_name}/{fading}_precoder_data_{num_ant}_ant_SNR_{SNR_dB}dB_{bit_codebook}_bit_codebk'
    print(file_name)
    with open(f'{file_name}.pkl', 'rb') as f:
        precoders, labels = load(f)
    return precoders, labels

## Visualization of Data

In [256]:
def visualize(precoders):
    # test plots
    for i in range(2):
        plt.scatter(precoders[i][0::2], precoders[i][1::2])
        plt.show()

## Precoder dataset class that packages the dataset into a format compatible with PyTorch

In [257]:
class PrecoderDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        return x, y
    
    def __len__(self):
        return len(self.data)

## Neural Network Class

In [258]:

class Model(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        # define layers and activation function as class parameters
        self.fc1 = nn.Linear(in_features=input_size, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=100)
        self.output = nn.Linear(in_features=100, out_features=output_size)
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        # define NN connections here, output of last layer is pred
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        pred = self.softmax(self.output(x))
        return pred


## Essential System Parameters

In [259]:
max_iter = 100000
fading = 1
num_ant = 2
SNR_dB = 20
bit_codebook = 1
visual = 0

## Training Parameters

In [260]:
learning_rate = 0.01
num_epochs = 100 # Number of times you go through the whole dataset

## Simulation iterations
1. Load dataset
2. Visualize, if necessary 
3. Create instance of precoder dataset class (Package dataset such that it is compatible for PyTorch)
4. Split dataset into 80% training and 20% validation
5. Create PyTorch dataloaders for training and validation data (Dataloaders divide the dataset into chuncks called batches. We can pass data to the neural network one batch at a time) 
6. Create an instance of neural network class
7. Define the loss and optimizer
8. Epoch - one pass of the entire dataset (training + validation)
9. In each epoch you iterate over several batches (Batch - chunk of data that is fed to the neural network in one go) 

In [ ]:
precoders, labels = load_dataset('datasets', fading, ant, SNR, bit)
if visual == 1:
    visualize(precoders)

precoder_data = stack(precoders, axis=0)
precoder_labels = array(labels)

precoder_dataset = PrecoderDataset(precoder_data, precoder_labels)
train, val = random_split(precoder_dataset, [int(0.8*max_iter), int(0.2*max_iter)])
train_loader = DataLoader(train, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(val, batch_size=128, shuffle=True, num_workers=0)

num_classes = 2**bit
nn_input_size = 2*ant*ant
nn_output_size = num_classes

model = Model(nn_input_size, nn_output_size)

if torch.cuda.is_available():
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = nn.CrossEntropyLoss()
for epoch in range(num_epochs):

    # TRAINING
    model.train()
    training_total_correct = 0
    training_losses = list()
    for batch in train_loader:

        x, y = batch # extracted from the batch

        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        # step 1: forward pass
        prob = model(x)

        output = prob
        target = y

        # step 2: compute objective function - measuring distance between the output of the network vs actual answer
        obj_func = loss(output, target)

        # step 3: clear the gradients
        model.zero_grad()

        # step 4: accumulate partial derivatives of obj_func wrt parameters
        obj_func.backward()

        # step 5: step in the opposite direction of the gradient
        optimizer.step()

        training_losses.append(obj_func.item())

        max_prob, max_ind = torch.max(output, 1)
        for ind in range(len(max_ind)):
            if torch.equal(max_ind[ind], y[ind]):
                training_total_correct += 1

    # VALIDATION
    model.eval()

    validation_total_correct = 0
    validation_losses = list()
    for batch in val_loader:

        x, y = batch # extracted from the batch

        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        # step 1: forward pass
        with torch.no_grad():
            prob = model(x)

        output = prob
        target = y

        # step 2: compute objective function - measuring distance between the output of the network vs actual answer
        obj_func = loss(output, target)
        validation_losses.append(obj_func.item())

        max_prob, max_ind = torch.max(output, 1)

        for ind in range(len(max_ind)):
            if torch.equal(max_ind[ind], y[ind]):
                validation_total_correct += 1
    training_accuracy = (training_total_correct/len(train))*100
    validation_accuracy = (validation_total_correct/len(val))*100

    print(f'Epoch {epoch + 1}, training loss: {torch.tensor(training_losses).mean():.8f}, training accuracy: {training_accuracy}%, validation loss: {torch.tensor(validation_losses).mean():.8f}, validation accuracy: {validation_accuracy[snr_loop, ant_loop, bit_loop, epoch]}%')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/utsa/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-261-55ddc41a7a3a>", line 1, in <module>
    precoders, labels = load_dataset('datasets', fading, ant, SNR, bit)
  File "<ipython-input-255-529c16c6c308>", line 4, in load_dataset
    os.makedirs(dir_name)
  File "/home/utsa/anaconda3/envs/py37/lib/python3.7/os.py", line 223, in makedirs
    mkdir(name, mode)
FileNotFoundError: [Errno 2] No such file or directory: 'datasets'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/utsa/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exc

## Dump validation results

In [ ]:
acc_pkl = 'validation_accuracy.pkl'
loss_pkl = 'validation_loss.pkl'

acc_pkl_file = open(acc_pkl, 'wb')
loss_pkl_file = open(loss_pkl, 'wb')

dump(validation_accuracy, acc_pkl_file)
dump(validation_loss_buffer, loss_pkl_file)

acc_pkl_file.close()
loss_pkl_file.close()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/utsa/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-261-24de0b76e7fe>", line 4, in <module>
    acc_pkl_file = open(acc_pkl, 'wb')
FileNotFoundError: [Errno 2] No such file or directory: 'validation_accuracy.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/utsa/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/utsa/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinne